# Fitting Evans2020 Experiment 1 data


In this notebook, we use PyBEAM's precoded sub-moudule to fit a Weibull changing threhsolds model to Evans2020's data. The citation for their publication is below:

    Evans, N. J., Hawkins, G. E., & Brown, S. D. (2020). The role of passing time in decision-making. Journal
    of Experimental Psychology: Learning, Memory, and Cognition, 46(2), 316–326. doi: 10.1037/xlm0000725. 
    
We specifically look at the Experiment 1 data from their publication. In that experiment, participants were given a random dot coherence task with four coherence conditions: 0%, 5%, 10%, and 40%. Reward rate was emphasized, with particpants instructed to maximize the number of correct decisions. Participants had a fixed time of 5 seconds in which to make the choice.

In addition to this notebook, three files located on the PyBEAM github are needed for this analysis: parseEvans2020.py, Evans2020_Exp1_data.csv, and Evans2020_Exp1_subjects.csv. We will discuss how they are used below.

First, we import PyBEAM's precoded sub-module. Additionally, we import parseEvans2020.py and numpy to handle data importing.


In [ ]:
# import PyBEAM
import pybeam.precoded as pbp

# import script to parse Evans data
from parseEvans2020 import subject_rt

# import numpy to load Evans data
import numpy as np


The next block of code imports and sorts the Evans Experiment 1 data set. We first load the data (Evans2020_Exp1_data.csv) and subject numbers (Evans2020_Exp1_subjects.csv) and place them in arrays. Data contains the participant number, choice made (correct or incorrect), coherence (0%, 5%, 10% or 40%), and reaction time. Subjects contains the list of participants.

To sort out the data for a single participant, we call the subject_rt function imported from parseEvans2020.py. It takes as inputs the data array, subjects arrays, the subject number we would like to fit, minimum rt value (filters out data below this value, as done in Evans), and the max rt value (filters out data above this value, as done in Evans).

subject_rt outputs a dictionary containing four keys: 'rt0', 'rt5', 'rt10', and 'rt40'. These contain dictionaries holding the reaction time data for each of the four coherence conditions. Each sub-dictionary contains two entries: 'rt_upper' and 'rt_lower'. 'rt_upper' contains an array of rt values corresponding to the correct choice. 'rt_lower' contains an array of values containing the rt values corresponding to an error. It outputs this particular structure becauase is the data structure required by PyBEAM's other functions.


In [ ]:
# import rt data from Evans2020 Experiment 1
data = np.asarray(np.loadtxt(open("Evans2020_Exp1_data.csv", "rb"), delimiter=",", skiprows=1))

# import subject numbers from Evans2020
subjects = np.asarray(np.loadtxt(open("Evans2020_Exp1_subjects.csv", "rb"), delimiter=",", skiprows=1))

# subject to fit model to (from list subjects)
subject_number = 31727

# filter data below min_rt and above max_rt
min_rt = 0.25
max_rt = 5.0

# file name to save posteriors to
file_name = 'subject_' + str(int(subject_number))

# return rt for desired subject number
rt = subject_rt(subject_number = subject_number, 
                          data = data, 
                      subjects = subjects, 
                        min_rt = min_rt, 
                        max_rt = max_rt)

rt


Now that we have imported our data, we now define our model. Following Evans, we use an EAM with Weibull thresholds and uniform contamination. If you are unsure how to use the model call out, see Tutorial 1 under "getting_started" on the PyBEAM github.


In [ ]:
# call changing thresholds model
model = pbp.changing_thresholds(contamination = 'uniform')


Now that we have defined our model, we can define our priors, conditions, and run the inference program. Since we have four coherences, we will have four model conditions, each with their own unique drift rate prior. Each model condition has its own data set, corresponding to one of the four coherences. Following Evans, we assume that the threshold will collapse completely from its initial location, so we set the collapse parameter, c, to -1 in the priors dictionary. We also assume that the uniform contamination can occur as any time between the min and max rt values, so we set the lower and upper contaminations to be theses values.


In [ ]:
# define priors
p = {'ptnd' : 'Uniform("tnd", lower = 0.0, upper = 0.75)',  # prior for non-decision time
       'pw' : 'Uniform("w", lower = 0.3, upper = 0.7)',      # prior for relative start
     'pmu0' : 'Uniform("mu0", lower = -5.0, upper = 5.0 )',  # drift rate prior for coherence 0%
     'pmu5' : 'Uniform("mu5", lower = -5.0, upper = 5.0)',   # drift rate prior for choherence 5%
    'pmu10' : 'Uniform("mu10", lower = -5.0, upper = 5.0)',  # drift rate prior for coherence 10%
    'pmu40' : 'Uniform("mu40", lower = -5.0, upper = 5.0)',  # drift rate prior for coherence 40%
       'pb' : 'Uniform("b", lower = 0.25, upper = 2.0)',     # prior for threshold location
    'plamb' : 'Uniform("lamb", lower = -1.0, upper = 2.0)',  # prior for scale parameter
   'pkappa' : 'Uniform("kappa", lower = -1.0, upper = 2.0)', # prior for shape parameter
        'c' : -1.0,                                          # collapse parameter value
       'pg' : 'Uniform("g", lower = 0.0, upper = 0.75)',     # prior for contamination strength
       'gl' : min_rt,                                        # uniform contamination lower bound
       'gu' : max_rt}                                        # uniform contamination upper bound

# model condition for coherence 0%
c0 = {'rt' : rt['rt0'],
     'tnd' : 'ptnd',
       'w' : 'pw',
      'mu' : 'pmu0',
       'b' : 'pb',
    'lamb' : 'plamb',
   'kappa' : 'pkappa',
       'c' : 'c',
       'g' : 'pg',
      'gl' : 'gl',
      'gu' : 'gu'}

# model condition for coherence 5%
c5 = {'rt' : rt['rt5'],
     'tnd' : 'ptnd',
       'w' : 'pw',
      'mu' : 'pmu5',
       'b' : 'pb',
    'lamb' : 'plamb',
   'kappa' : 'pkappa',
       'c' : 'c',
       'g' : 'pg',
      'gl' : 'gl',
      'gu' : 'gu'}

# model condition for coherence 10%
c10 = {'rt' : rt['rt10'],
       'tnd' : 'ptnd',
         'w' : 'pw',
        'mu' : 'pmu10',
         'b' : 'pb',
      'lamb' : 'plamb',
     'kappa' : 'pkappa',
         'c' : 'c',
         'g' : 'pg',
        'gl' : 'gl',
        'gu' : 'gu'}

# model condition for coherence 40%
c40 = {'rt' : rt['rt40'],
      'tnd' : 'ptnd',
        'w' : 'pw',
       'mu' : 'pmu40',
        'b' : 'pb',
     'lamb' : 'plamb',
    'kappa' : 'pkappa',
        'c' : 'c',
        'g' : 'pg',
       'gl' : 'gl',
       'gu' : 'gu'}

# load conditions into dictionary
cond = {0 : c0, 1 : c5, 2 : c10, 3 : c40}

# run parameter inference
idata = pbp.inference(model = model,
                     priors = p,
                 conditions = cond,
                    samples = 50000,
                     chains = 3,
                      cores = 3,
                  file_name = file_name)


In [ ]:
# plot posteriors
pbp.plot_idata(file_name = file_name, burnin = 25000);


In [ ]:
# posterior summary statistics
pbp.summary(file_name = file_name, burnin = 25000)
